# Data Cleaning

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

Specifically, we'll be walking through:

1. **Getting the data - **in this case, we'll be scraping data from a website
2. **Cleaning the data - **we will walk through popular text pre-processing techniques
3. **Organizing the data - **we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Problem Statement

As a reminder, our goal is to look at transcripts of various comedians and note their similarities and differences. Specifically, I'd like to know if Ali Wong's comedy style is different than other comedians, since she's the comedian that got me interested in stand up comedy.

## Getting The Data

Luckily, there are wonderful people online that keep track of stand up routine transcripts. [Scraps From The Loft](http://scrapsfromtheloft.com) makes them available for non-profit and educational purposes.

To decide which comedians to look into, I went on IMDB and looked specifically at comedy specials that were released in the past 5 years. To narrow it down further, I looked only at those with greater than a 7.5/10 rating and more than 2000 votes. If a comedian had multiple specials that fit those requirements, I would pick the most highly rated one. I ended up with a dozen comedy specials.

In [24]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    trump = ''
    biden = ''
    wallace = ''
    for p in soup.find(class_="fl-callout-text").find_all('p'):
        if p.text.split():
            name = p.text.split()[0]
            if name == 'Chris':
                wallace += p.text
            elif name == 'President':
                trump += p.text
            elif name == 'Vice':
                biden += p.text
            
    return trump,biden,wallace

# Speakers
speakers = ['Trump','Biden','Wallace']

In [25]:
# Transcript the speech, and assigning the speech to each speaker. 
url = 'https://www.rev.com/blog/transcripts/donald-trump-joe-biden-1st-presidential-debate-transcript-2020'
trump,biden,wallace = url_to_transcript(url)
transcripts = []
transcripts.append(trump)
transcripts.append(biden)
transcripts.append(wallace)

In [26]:
mkdir transcripts

Sottodirectory o file transcripts gi… esistente.


In [27]:
# # Pickle files for later use

# # Make a new directory to hold the text files
# !mkdir transcripts

for i, c in enumerate(speakers):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)

In [28]:
# Load pickled files
data = {}
for i, c in enumerate(speakers):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [29]:
# Double check to make sure data has been loaded properly
data.keys()

dict_keys(['Trump', 'Biden', 'Wallace'])

In [30]:
# More checks
data['Trump']

'President Donald J. Trump: (02:51)\nHow are you doing?President Donald J. Trump: (04:01)\nThank you very much, Chris. I will tell you very simply. We won the election. Elections have consequences. We have the Senate, we have the White House, and we have a phenomenal nominee respected by all. Top, top academic, good in every way. Good in every way. In fact, some of her biggest endorsers are very liberal people from Notre Dame and other places. So I think she’s going to be fantastic. We have plenty of time. Even if we did it after the election itself. I have a lot of time after the election, as you know. So I think that she will be outstanding. She’s going to be as good as anybody that has served on that court. We really feel that. We have a professor at Notre Dame, highly respected by all, said she’s the single greatest student he’s ever had. He’s been a professor for a long time at a great school.President Donald J. Trump: (04:53)\nAnd we won the election and therefore we have the rig

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [31]:
# Let's take a look at our data again
next(iter(data.keys()))

'Trump'

In [32]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

'President Donald J. Trump: (02:51)\nHow are you doing?President Donald J. Trump: (04:01)\nThank you very much, Chris. I will tell you very simply. We won the election. Elections have consequences. We have the Senate, we have the White House, and we have a phenomenal nominee respected by all. Top, top academic, good in every way. Good in every way. In fact, some of her biggest endorsers are very liberal people from Notre Dame and other places. So I think she’s going to be fantastic. We have plenty of time. Even if we did it after the election itself. I have a lot of time after the election, as you know. So I think that she will be outstanding. She’s going to be as good as anybody that has served on that court. We really feel that. We have a professor at Notre Dame, highly respected by all, said she’s the single greatest student he’s ever had. He’s been a professor for a long time at a great school.President Donald J. Trump: (04:53)\nAnd we won the election and therefore we have the rig

In [37]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text,key):
    word_to_clean = ''
    if key == 'Trump': 
        word_to_clean = 'President Donald J. Trump'
    elif key == 'Biden':
        word_to_clean =  'Vice President Joe Biden'
    else:
        word_to_clean = 'Chris Wallace'
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ''.join(list_of_text)
    #cleaning word
    combined_text = combined_text.replace(word_to_clean,'')
    return combined_text

In [38]:
# Combine it!
data_combined = {key: [combine_text(value,key)] for (key, value) in data.items()}

In [39]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
Biden,": (02:49)\nHow you doing, man?: (02:51)\nI’m well.: (05:29)\nWell, first of all, thank you for doing this and looking forward to this, Mr. Preside..."
Trump,": (02:51)\nHow are you doing?: (04:01)\nThank you very much, Chris. I will tell you very simply. We won the election. Elections have consequences...."
Wallace,: (01:20)\nGood evening from the Health Education Campus of Case Western Reserve University and the Cleveland Clinic. I’m of Fox News and I welco...


In [40]:
# Let's take a look at the transcript for Ali Wong
data_df.transcript.loc['Trump']

': (02:51)\nHow are you doing?: (04:01)\nThank you very much, Chris. I will tell you very simply. We won the election. Elections have consequences. We have the Senate, we have the White House, and we have a phenomenal nominee respected by all. Top, top academic, good in every way. Good in every way. In fact, some of her biggest endorsers are very liberal people from Notre Dame and other places. So I think she’s going to be fantastic. We have plenty of time. Even if we did it after the election itself. I have a lot of time after the election, as you know. So I think that she will be outstanding. She’s going to be as good as anybody that has served on that court. We really feel that. We have a professor at Notre Dame, highly respected by all, said she’s the single greatest student he’s ever had. He’s been a professor for a long time at a great school.: (04:53)\nAnd we won the election and therefore we have the right to choose her, and very few people knowingly would say otherwise. And by

In [41]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)


In [42]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
Biden,\nhow you doing man \ni’m well \nwell first of all thank you for doing this and looking forward to this mr president \nthe american people have a...
Trump,\nhow are you doing \nthank you very much chris i will tell you very simply we won the election elections have consequences we have the senate we...
Wallace,\ngood evening from the health education campus of case western reserve university and the cleveland clinic i’m of fox news and i welcome you to...


In [43]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [44]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
Biden,how you doing man im well well first of all thank you for doing this and looking forward to this mr president the american people have a right to...
Trump,how are you doing thank you very much chris i will tell you very simply we won the election elections have consequences we have the senate we hav...
Wallace,good evening from the health education campus of case western reserve university and the cleveland clinic im of fox news and i welcome you to th...


**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [45]:
# Let's take a look at our dataframe
data_df

,transcript
Biden,": (02:49)\nHow you doing, man?: (02:51)\nI’m well.: (05:29)\nWell, first of all, thank you for doing this and looking forward to this, Mr. Preside..."
Trump,": (02:51)\nHow are you doing?: (04:01)\nThank you very much, Chris. I will tell you very simply. We won the election. Elections have consequences...."
Wallace,: (01:20)\nGood evening from the Health Education Campus of Case Western Reserve University and the Cleveland Clinic. I’m of Fox News and I welco...


In [49]:
# Let's add the comedians' full names as well
full_names = ['President Donald J. Trump','Vice President Joe Biden','Chris Wallace']

data_clean['full_name'] = full_names
data_clean

,transcript,full_name
Biden,how you doing man im well well first of all thank you for doing this and looking forward to this mr president the american people have a right to...,President Donald J. Trump
Trump,how are you doing thank you very much chris i will tell you very simply we won the election elections have consequences we have the senate we hav...,Vice President Joe Biden
Wallace,good evening from the health education campus of case western reserve university and the cleveland clinic im of fox news and i welcome you to th...,Chris Wallace


In [50]:
# Let's pickle it for later use
data_clean.to_pickle("corpus.pkl")

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [51]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,ability,able,abolishing,abraham,absolutely,absorbed,abuse,academic,accept,accepted,...,years,yes,york,youd,youll,young,younger,youre,youve,zero
Biden,2,17,0,0,3,1,0,0,2,3,...,3,5,0,1,0,1,0,10,2,1
Trump,0,1,0,0,3,0,0,1,1,0,...,18,5,4,2,6,2,1,19,13,0
Wallace,0,1,1,1,0,0,1,0,0,0,...,14,3,0,2,2,0,0,10,6,1


In [52]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [54]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

## Additional Exercises

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?